In [14]:
# !pip install chromadb pandas

In [15]:
# !pip install langchain

In [19]:
# !pip install sentence-transformers

In [18]:
# !pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [20]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

In [21]:
# 1. 데이터 로드
cosmetics_df = pd.read_csv("cosmetics.csv")
ingredients_df = pd.read_csv("ingredients.csv")

In [6]:
cosmetics_df.head()

,category,brand,product_name,sale_price,usage,ingredient,reviews
0,스킨/토너,라네즈,라네즈 크림스킨,"26,400","1. 기본 사용법: 아침, 저녁 스킨 사용 단계에서 적당량을 손에 덜어 피부 속부터...","정제수,부틸렌글라이콜,글리세린,메도우폼씨오일,아이소펜틸다이올,1,2-헥산다이올,스쿠...",['상품은 너무 좋아요. 근데 제발 환경파괴하는 이런 사은품은 그만 만들었으면 좋겠...
1,스킨/토너,아누아,아누아 어성초 수딩 토너,"31,500",적당량을 덜어 피부에 골고루 펴 발라줍니다.,"약모밀꽃/잎/줄기수,정제수,1,2-헥산다이올,글리세린,베타인,부틸렌글라이콜,아이소펜...",['순하고 무난한 토너입니다. 어성초 성분이 들어있어 진정에도 도움되는 거 같아 트...
2,스킨/토너,라운드랩,라운드랩 1025 독도 토너,"27,000","세안 후 적당량을 덜어 화장솜에 충분히 적신 뒤, 피부결을 따라 부드럽게 닦아내어 ...","정제수,부틸렌글라이콜,글리세린,펜틸렌글라이콜,프로판다이올,아이리쉬모스추출물,사탕수수...",['올리브영 알바생의 라운드랩 독토 토너 .닦토 수분 충전 저는 개인적으로 닦토...
3,스킨/토너,더랩바이블랑두,더랩바이블랑두 올리고 히알루론산 딥 토너,"28,900",적당량을 손에 덜어 피부결을 따라 발라주고 손 끝으로 가볍게 두드려 흡수시켜 줍니다...,"정제수,부틸렌글라이콜,1,2-헥산다이올,글리세린,베타-글루칸,베타인,소듐하이알루로네...",['제형이 되게 가벼워서 속수분이 채워질까 궁금했는데 가벼운 제형과는 달리 속건조가...
4,스킨/토너,라로슈포제,라로슈포제 똘러리앙 울트라 로션,"27,360","아침 저녁, 세안 후 스킨 토너 단계에서 적당량을 화장솜에 덜어 얼굴 전체에 부드럽...",정제수 글리세린 다이프로필렌글라이콜 부틸렌글라이콜 프로필렌글라이콜 펜틸렌글라이콜 나...,['자극 없이 순하고 스킨팩할때 꼭 사용하는 로션인데 하고 나서도 건조함 없이 촉촉...


In [7]:
ingredients_df.head()

,ingredient,description
0,힙시지구스 울마리우스균사체추출물,"힙시지구스 울마리우스균사체추출물은 피부 보습과 진정을 돕는 성분으로, 항산화 효과가..."
1,힙네아 무스키포르미스추출물,"힙네아 무스키포르미스 추출물은 홍조류에서 추출된 성분으로, 보습 및 피부 진정 효과..."
2,힙티스 수아베올렌스씨오일,히프티스 수아베올렌스씨오일은 피부 보습과 진정에 도움을 줄 수 있는 식물성 오일입니...
3,히푸릭애씨드,히푸릭애씨드(Hippuric Acid)는 주로 항산화 및 항균 작용을 가지고 있어 ...
4,"히오스신, 그 염류 및 유도체","히오스신, 그 염류 및 유도체는 일반적으로 진정 및 항콜린 작용을 하는 화합물입니다..."


In [9]:
cosmetics_df.describe()

,category,brand,product_name,sale_price,usage,ingredient,reviews
count,4366,4366,4366,4366,4366,4366,4366
unique,22,468,3701,643,2907,3574,4310
top,에센스/세럼/앰플,라운드랩,바이오더마 센시비오 H2O,"28,000",본 품 적당량을 취해 피부에 골고루 펴 바른다.,"정제수,글리세린,미리스틱애씨드,팔미틱애씨드,포타슘하이드록사이드,글라이콜다이스테아레이...",['다써서 또 샀습니다 번 재구매템. 메디힐 제품과 피부가 잘 맞는듯 해서 좋으네요...
freq,652,81,7,87,32,7,3


In [8]:
ingredients_df.describe()

,ingredient,description
count,23544,23544
unique,23540,23544
top,"석유 증류물, hydrodesulfurised light catalytic cracked","""눈""은 화장품 성분이 아니며, 얼굴 기관 중 하나로 여러 화장품에서 효과를 높이기..."
freq,2,1


In [22]:
# 2. 임베딩 모델 준비
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

<ipython-input-22-8985b7b4a1c4>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
# 3. CSV -> LangChain Document로 변환
documents = []

In [24]:
# 성분 문서
for _, row in ingredients_df.iterrows():
    content = row['description']
    metadata = {
        "type": "ingredient",
        "name": row['ingredient']
    }
    documents.append(Document(page_content=content, metadata=metadata))

In [26]:
# 화장품 문서
for _, row in cosmetics_df.iterrows():
    content = f"제품명: {row['product_name']}, 성분: {row['ingredient']}, 리뷰: {row['reviews']}, 사용법: {row['usage']}"
    metadata = {
        "type": "cosmetic",
        "product_name": row['product_name']
    }
    documents.append(Document(page_content=content, metadata=metadata))

In [25]:
documents[0]

Document(metadata={'type': 'ingredient', 'name': '힙시지구스 울마리우스균사체추출물'}, page_content='힙시지구스 울마리우스균사체추출물은 피부 보습과 진정을 돕는 성분으로, 항산화 효과가 있어 피부의 건강과 생기를 유지하는 데 기여합니다. 또한 피부 장벽 강화에 도움을 주어 외부 자극으로부터 피부를 보호할 수 있습니다.')

In [28]:
len(documents)

27910

In [ ]:
# 4. Chroma 벡터 DB로 저장
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

vectordb.persist()
print("ChromaDB에 저장 완료")

In [ ]:
import pandas as pd
from transformers import pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
# ChromaDB 로드
chromadb = Chroma(
		persist_directory="./chroma_db",
		embedding_function=embedding_model
)

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [ ]:
# RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True
)

# 6. 질문 실행
query = "잡티 제거에 좋은 화장품은?"
result = qa_chain({"query": query})

print("🧠 답변:", result["result"])
print("📄 사용된 문서:", result["source_documents"])